## 扩展内置类型

### 通过内嵌方式扩展类型

In [1]:
class Set:
    def __init__(self, value=[]):
        self.data = []
        self.concat(value)

    def intersect(self, other):
        res = []
        for x in self.data:
            if x not in other:
                res.append(x)
        return Set(res)

    def union(self, other):
        res = self.data[:]
        for x in other:
            if x not in res:
                res.append(x)
        return Set(res)
    
    def concat(self, value):
        for x in value:
            if not x in self.data:
                self.data.append(x)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, key):
        return self.data[key]

    def __and__(self, other):
        return self.intersect(other)
    
    def __or__(self, other):
        return self.union(other)
    
    def __repr__(self):
        return 'Set:' + repr(self.data)
    
    def __iter__(self):
        return iter(self.data)

In [4]:
x = Set([1, 3, 5, 7])
print(x.union(Set([1, 4, 7])))

Set:[1, 3, 5, 7, 4]


### 通过子类扩展类型

In [6]:
class Mylist(list):
    def __getitem__(self, offset):
        print('(indexing %s at %s)' % (self, offset))
        return list.__getitem__(self, offset-1)

In [7]:
x = Mylist('abc')
print(x[1])

(indexing ['a', 'b', 'c'] at 1)
a


## 新式类

在python3中所有的类都是新式类，不管它们是否显示的继承自object，所有类都继承自object。但是python2中只有新式类继承自object。

### 内置类型的获取将跳过实例

通用实例属性拦截方法`__getattr__`和`__getattribute__`不再拦截下以`__X__`命名得运算符重载方法的调用，这类方法从类开始，而不是实例。

如一个类重载了`__getitem__`，它的一个实例为`X`。那么X[i]在经典类中是`x.__getitem__[i]`,在新式类中则是`type(X).__getitem__(X, i)`

In [55]:
class C(object):
    data = 'spam'
    def __getattr__(self, name):
        print(name)
        return getattr(self.data, name)

    def __len__(self):
        return 10

In [56]:
X = C()
X[0] # 这种委托模式，data是有__getitem__的，但是通过getattr并不能被获取到

TypeError: 'C' object is not subscriptable

In [57]:
X.upper() # 非__开头结尾的就可以拦截到

upper


'SPAM'

In [53]:
s = 'sapm'
s.upper()

'SAPM'

In [58]:
# 由于这类方法的搜索不是从实例开始的，所以在实例上重载也没用
X.__add__ = lambda y:88 + y 
X + 1

SyntaxError: invalid syntax (<ipython-input-58-25530fab3493>, line 1)

In [60]:
len(x) # 通过在类中重载则有用，因为会直接取类里面搜索

3

In [61]:
class C1:pass

In [62]:
I = C1()

In [66]:
type(I), I.__class__, C1.__class__

(__main__.C1, __main__.C1, type)

In [65]:
type(type)

type

所以新式类就要求，为普通方法名定义`__getattr__`，为内置运算重新定义

### 类型模型改变

类和类型是一样的，类是type对象的实例，而类产生它们自己实例。两者都可以视为类型，因为它们都生成实例。类是类型，类型也是类

In [69]:
isinstance(list, object), isinstance(dict, object), isinstance(C1, object)

(True, True, True)

In [71]:
type(type), type(object), isinstance(type, object) # 这里有点晕哟

(type, type, True)

In [72]:
dir(object) # 全是内置类型啊

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

### 钻石继承

当多个父类继承于同一个父类时，形状类似钻石，所以称为钻石继承，新式类中采用广度优先的继承顺序

In [1]:
class A: attr = 1
class B(A): pass
class C(A): attr = 3
class D(B, C): pass 
class E(B, C): attr = B.attr

In [2]:
x = D()
y = E()
x.attr, y.attr

(3, 1)

上面的例子中x是D的实例，其中搜索顺序是D->B->C->A，结果在C中找到了

y是E的实例，由于在E中指定了属性，搜索顺序就是 D->B->A, 结果找到了A中的值

In [3]:
D.__mro__ # 搜索顺序

(__main__.D, __main__.B, __main__.C, __main__.A, object)

## 新式类扩展

### slot 属性声明

需要在class语句内将字符串名称顺序赋值给特殊变量`__slots__`，只有`__slots__`列表内的这些名称可赋值为实例属性。在引用前必须先赋值

In [4]:
class limiter(object):
    __slots__ = ['age', 'name', 'job']

In [5]:
x = limiter()

In [6]:
x.age # 未赋值

AttributeError: age

In [7]:
x.age = 40
x.age

40

In [8]:
x.aaa = 40 # 不再slot内不能赋值

AttributeError: 'limiter' object has no attribute 'aaa'

slot会使类模型复杂化，同时会复杂化那些依赖于slot的代码。

使用了slot后，实例会失去`__dict__`命名字典空间,但是仍然可以用getattr, setattr, dir()访问

In [10]:
x.__dict__

AttributeError: 'limiter' object has no attribute '__dict__'

In [14]:
setattr(x, 'age', 50)
getattr(x, 'age')

50

如果没有了属性命名空间字典，将不能给不在slot列表中的实例赋值新的名称

In [18]:
class D:
    __slots__ = ['a', 'b']
    def __init__(self):
        self.d = 10

In [19]:
x = D() # d不在slots中，不能创建该属性

AttributeError: 'D' object has no attribute 'd'

手动加入 `__dict__`, 就可以添加自定义属性了

In [21]:
class D:
    __slots__ = ['a', 'b', '__dict__']
    c = 50
    def __init__(self):
        self.d = 10

In [27]:
x = D()
x.c, x.d, x.__dict__, D.__dict__, dir(D)

(50,
 10,
 {'d': 10},
 mappingproxy({'__module__': '__main__',
               '__slots__': ['a', 'b', '__dict__'],
               'c': 50,
               '__init__': <function __main__.D.__init__(self)>,
               'a': <member 'a' of 'D' objects>,
               'b': <member 'b' of 'D' objects>,
               '__dict__': <attribute '__dict__' of 'D' objects>,
               '__doc__': None}),
 ['__class__',
  '__delattr__',
  '__dict__',
  '__dir__',
  '__doc__',
  '__eq__',
  '__format__',
  '__ge__',
  '__getattribute__',
  '__gt__',
  '__hash__',
  '__init__',
  '__init_subclass__',
  '__le__',
  '__lt__',
  '__module__',
  '__ne__',
  '__new__',
  '__reduce__',
  '__reduce_ex__',
  '__repr__',
  '__setattr__',
  '__sizeof__',
  '__slots__',
  '__str__',
  '__subclasshook__',
  'a',
  'b',
  'c'])

### 父类中的多个slots列表

如果多个父类中都有slots列表的话，实例按照一般继承规则，获得类树中其他位置所有slot名称的并集

In [28]:
class E:
    __slots__ = ['c', 'd']

In [29]:
class D(E):
    __slots__ = ['a', '__dict__']

In [32]:
x = D()
x.a = 1
x.b = 2
x.c = 3
x.a, x.b, x.c, x.__slots__ # 可见它的slot并不包括父类的

(1, 2, 3, ['a', '__dict__'])

### 使用规则

- 父类中没有slot，子类中的将没有意义，因为会继承父类的`__dict__`
- 如果子类没有slot，那么父类的也没意义，因为它会创建
- 在slots中重新定义同名的属性，会让父类的没有意义
- slot会组织类一级的默认名称

## Porperty 属性访问器

Porperty 是一种被赋值给类属性名称的对象。porperty对于所有的类都可用,但对于拦截属性赋值则必须要求新式类才有效

In [2]:
class properties(object):
    def getage(self):
        return 40
    
    def setage(self, value):
        print('set age:%s' % value)
        self.age = value 

    # 参数为三个访问器方法,获取，设置和删除
    age = property(getage, setage, None, None)

In [8]:
x = properties()
# x.age = 42
x.age, x.__dict__

(40, {})

In [7]:
x.name

AttributeError: 'properties' object has no attribute 'name'

In [10]:
x.job = 'lalla'
x.job, x.__dict__

('lalla', {'job': 'lalla'})

### `__getattribute__`和描述符

`__getattribute__`能拦截所有属性的引用，而不局限于未定义的引用。除此之外，python还有专业化的属性拦截任务支持了属性描述符的概念。描述符带有`__get__`和`__set__`方法的类，被赋值给类属性并且能被实例继承，拦截对特定属性的读取和访问。

In [11]:
class AgeDesc(object):
    def __get__(self, instance, owner): 
        return 40
    
    def __set__(self, instance, value):
        instance._age = value 

In [12]:
class descriptors(object):
    age = AgeDesc()

In [13]:
x = descriptors()
x.age

40

In [14]:
x.age = 42
x._age 

42

## 静态方法和类方法

静态方法大致与类中简单的无实例函数的工作方式类似，而类方法则被传入一个类而不是一个实例。  
静态方法是嵌套在一个类中的没有self参数的简单函数，并且旨在操作类属性而不是实例属性。静态方法不会接受一个自动的self参数，不管是通过一个类还是一个实例被调用。它们通常会记录横跨所有实例的信息，而不是为实例提供行为。  
类方法是类的一种方法，传入给它们的第一位参数是一个类对象而不是一个实例对象，不管是通过一个实例或一个类调用它们。

In [1]:
class Spam:
    # 所有实例共享的属性
    numInstances = 0
    def __init__(self):
        Spam.numInstances = Spam.numInstances + 1
    def printNumInstances():
        print('Number of instance created: ', Spam.numInstances)

In [2]:
a = Spam()
b = Spam()
c = Spam()

In [5]:
Spam.printNumInstances()

Number of instance created:  3


In [6]:
c.printNumInstances() # 不能通过实例调用，因为没有接受self的参数

TypeError: printNumInstances() takes 0 positional arguments but 1 was given

### 使用静态方法和类方法

In [7]:
class Methods:
    # 实例方法
    def imeth(self, x):
        print([self, x])
    
    def smeth(x):
        print([x])
    
    def cmeth(cls, x):
        print([cls, x])
        
    # 静态方法
    smeth = staticmethod(smeth)
    # 类方法
    cmeth = classmethod(cmeth)

### 使用静态方法计数实例

In [8]:
class Spam:
    # 所有实例共享的属性
    numInstances = 0
    def __init__(self):
        Spam.numInstances = Spam.numInstances + 1

    def printNumInstances():
        print('Number of instance created: ', Spam.numInstances)
    printNumInstances = staticmethod(printNumInstances)

In [9]:
a = Spam()
b = Spam()
c = Spam()

In [12]:
Spam.printNumInstances(),
c.printNumInstances()

Number of instance created:  3
Number of instance created:  3


使用静态可以把函数名变成类作用域内的局部名称；而且把函数代码移到靠近其使用的地方；同时还能让子类通过继承来定制静态方法。

In [13]:
class other(Spam):
    pass 

In [14]:
c = other()
c.numInstances # 继承的话会跟父类共享同一个变量

4

### 用类方法计数实例

In [15]:
class Spam:
    # 所有实例共享的属性
    numInstances = 0
    def __init__(self):
        Spam.numInstances = Spam.numInstances + 1

    def printNumInstances(cls):
        print('Number of instance created: ', Spam.numInstances)
    printNumInstances = classmethod(printNumInstances)

In [16]:
a, b = Spam(), Spam()

In [18]:
a.printNumInstances()
Spam.printNumInstances()

Number of instance created:  2
Number of instance created:  2


## 装饰器和元类1

函数装饰器：用来增强函数定义，同时为简单函数和类方法指明了特殊运算模式，也就是通过把简单函数和类方法包在一层额外的逻辑实现中，因而也常被称为一个元函数。  
类装饰器：用来增强类定义，同样为类添加管理全体对象和其接口的支持。  
函数装饰器可以被看作是它后边跟着的函数的运行时的声明。只需要单独一行，就写在函数或方法的def语句之前，包括@和元函数。效果类似于将函数传递给装饰器，并再赋值回最初的函数名。

In [4]:
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func 
    
    def __call__(self, *args):
        self.calls += 1
        print('call %s to %s ' % (self.calls, self.func.__name__))
        return self.func(*args)

In [5]:
@tracer
def spam(a, b, c):
    return a + b + c 

In [6]:
print(spam(1, 2, 3))
print(spam('a', 'b', 'c'))

call 1 to spam 
6
call 2 to spam 
abc


In [7]:
@tracer
def addd(a, b):
    return a + b

In [8]:
print(addd(1, 2))

call 1 to addd 
3


因为spam函数是通过tracer装饰器执行的，所以当原来的函数名spam被调用时，实际上触发的是类中的`__call__`方法。这个方法会计数并记录该次调用，然后委托给被包装的原来的函数。

In [10]:
def tracer2(func):
    def oncall(*args):
        oncall.calls += 1
        print('call %s to %s' %(oncall.calls, func.__name__))
        return func(*args)
    oncall.calls = 0
    return oncall

In [21]:
@tracer2
def addd(a, b):
    return a + b
# 等价于 tracer2(addd)(a, b)

In [12]:
print(addd(1, 2))

call 1 to addd
3


In [24]:
tracer2(addd)(1, 2)

call 1 to oncall
call 3 to addd


3

In [19]:
class C:
    @tracer2
    def spam(self, a, b):
        return a + b

In [20]:
x = C()
print(x.spam(1, 2))
print(x.spam(4, 5))

call 1 to spam
3
call 2 to spam
9


## super内置函数

内置super函数能够通用地调用父类地方法

In [25]:
class C:
    def act(self):
        print('spam')

In [26]:
class D(C):
    def act(self):
        super().act()
        print('eggs')

In [27]:
x = D()
x.act()

spam
eggs


采用了super后，如果未来父类发生了改变，不需要更改代码。但是super通过检查调用栈来自动定位self参数和寻找父类，并且将self参数和父类配对在一个特殊地代理对象中，从而将之后地调用路由到父类地方法版本中。

但是当多继承时就会引发问题：

In [32]:
class A:
    def act(self):
        print('A')

class B:
    def act(self):
        print('B')

class C(A, B):
    def act(self):
        super().act()

In [33]:
x = C()
x.act()

A


在多继承中，super总会取出左边第一个带有这一方法地类，可能并不是想要的。

### super的优势：继承树的修改和分发

- 在运行时改变类树：就是在子类中不硬编码父类的方法调用
- 协同所继承方法的分发

### 协同所继承方法的分发

In [35]:
class A:
    def __init__(self):
        print('A.__init__')

class B(A):
    def __init__(self):
        print('B.__init__')
        A.__init__(self)

class C(A):
    def __init__(self):
        print('C.__init__')
        A.__init__(self)

class D(B, C):
    def __init__(self):
        B.__init__(self)
        C.__init__(self)

In [36]:
x = D() # 可见A的构造函数调用两次

B.__init__
A.__init__
C.__init__
A.__init__


In [39]:
class A:
    def __init__(self):
        print('A.__init__')

class B(A):
    def __init__(self):
        print('B.__init__')
        super().__init__()

class C(A):
    def __init__(self):
        print('C.__init__')
        super().__init__()

class D(B, C):
    pass 

In [40]:
x = D()

B.__init__
C.__init__
A.__init__


In [41]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

只要所有的类都采用了super调用，通过在MRO序列中选择下一个类，一个类方法中的super调用就能在类树上传递调用。D调用了B，B转发调用C，C调用A。如果没有都是用super调用，B就无法转发调用。

In [42]:
class A:
    def __init__(self):
        print('A.__init__')

class B(A):
    def __init__(self):
        print('B.__init__')
        super().__init__()

class C(A):
    def __init__(self):
        print('C.__init__')

class D(B, C):
    pass 

In [43]:
c =D()

B.__init__
C.__init__


In [44]:
class A:
    def __init__(self):
        print('A.__init__')

class B(A):
    def __init__(self):
        print('B.__init__')

class C(A):
    def __init__(self):
        print('C.__init__')
        super().__init__()

class D(B, C):
    pass 

In [45]:
x = D()

B.__init__


## 类陷阱

- 修改类属性可能会造成副作用
- 修改可变类属性也坑你造成副作用
- 多继承：顺序很重要
- 方法和类中的作用域
